<a href="https://colab.research.google.com/github/tluxxx/weekly-pattern-in-stock-markets/blob/main/weekly_patterns_(part_1_16_week_cycle_with_pattern_according_to_Gebert).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploiting weekly patterns in stock market for trading
#### part 1: strategy-idea originally generated by Thomas Gebert (published in: Kurzfrist-Strategien fuer Anleger, Boersenbuchverlag, 2019 (in German))

#1. Preparation, Data Upload and Definition of Helper Functions

In [ ]:
# preparation
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
# importing all modules
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt

In [4]:
# direct download price dataframe from yfinance
end_date = '2024-03-02' # dt.datetime.now()
start_date = '2000-01-01'
price = yf.download('^GDAXI', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


# 2. Helper Functions Code

In [18]:
# preparation: resampling daily to weekly data, numbering weeks (original strategy (first trading day at Close)
def transforming_daily_weekly_close(price):
  price['date'] = price.index
  df_input = {'start_w': price.date.resample('W').first(),
              'price': price.Close.resample('W').first()}
  price_w = pd.DataFrame(df_input)
  price_w = price_w.assign(week_nb=range(len(price_w)))
  price_w.set_index('start_w', inplace=True)
  return price_w

# preparation: resampling daily to weekly data, numbering weeks (modified strategy (first trading day at Open)
def transforming_daily_weekly_open(price):
  price['date'] = price.index
  df_input = {'start_w': price.date.resample('W').first(),
              'price': price.Open.resample('W').first()}
  price_w = pd.DataFrame(df_input)
  price_w = price_w.assign(week_nb=range(len(price_w)))
  price_w.set_index('start_w', inplace=True)
  return price_w

# preparation: matching weekly date with weekly pattern in a cycle
def pos_weekly(price_w, pos, shift):
  pos_dict = dict(enumerate(pos))
  cycle = len(pos)
  # calculation of week-type and resulting positions depending on the shift
  price_w['week_type'] = (price_w['week_nb'] + shift) % cycle
  price_w['pos'] = price_w['week_type'].map(pos_dict)


In [14]:
# analyses: calculation of pnl for a given weekly pattern (including positions)
def pnl_calculation(price_w):
  # calculation of PnL ** strategy
  rets = price_w['price'].pct_change()*price_w['pos'].shift(1)
  pnl_strategy = (1 + rets).cumprod()
  pnl_strategy2 = rets.cumsum() + 1
  # calculation of PnL ** buy and hold
  pnl_buyhold = (1 + price_w['price'].pct_change()).cumprod()
  # generation of results dataframe
  input = {'pnl_strategy':pnl_strategy*100, 'pnl_strategy2':pnl_strategy2*100, 'pnl_b&h':pnl_buyhold*100}   # pruefen
  pnl = pd.DataFrame(input)
   # calculating the final results
  strat = pnl['pnl_strategy'][-1]
  strat2 = pnl['pnl_strategy2'][-1]
  buh = pnl['pnl_b&h'][-1]
  return pnl, buh, strat, strat2

# anylyses: plotting of pnl over time
def pnl_plotting(pnl, title):
  fig = px.line(pnl, x=pnl.index, y=['pnl_strategy','pnl_strategy2','pnl_b&h'])
  fig.update_layout(title=title)
  fig.update_layout(autosize=False,width=800,height=400)
  fig.update_layout(xaxis_title="Date", yaxis_title='PnL in %')
  fig.show()

# 3. Original Strategy by Gebert (slightly simplified)
### Features:  
*   16-weeks cycle
*   tradetime for opening positions (buy/short) & closing positions (sell/cover): each monday 17:30 (at close)

### Modifications:
*   Original Strategy: If first trading day of a week is not Monday, trading on the last trading day of the previous week
*   Proposed Simplification: if first trading day of a week is not Monday, trading on the first trading day of that particular week


In [39]:
# original gebert pattern
pos_original = [0,0,0,0,0,0,0,-1,0,0,-1,0,1,1,1,-1]
# transforming daily into weekly data
price_w = transforming_daily_weekly_close(price)
# other parameters
shift_weeks = 12
pos = pos_original
# weekly trading positions
pos_weekly(price_w, pos, shift_weeks)

In [22]:
# check if 2018-11-12 is in week_type 4
price_w[price_w.index >'2018-11-01'].head(3)

,price,week_nb,week_type,pos
start_w,,,,
2018-11-05,11494.959961,983,3,0
2018-11-12,11325.440430,984,4,0
2018-11-19,11244.540039,985,5,0


In [23]:
# calculation of pnl time-series and final results
pnl, buh, strat, strat2 = pnl_calculation(price_w)
# plotting of pnl-timeseries and final results
title='PnL 16-week-cycle (orig.pattern) pattern vs. b&h Instrument: DAX [2000-2024]'
pnl_plotting(pnl, title)
print(f'Buy and Hold:   {buh:.2f} %   Strategy:  {strat:.2f} %    Strategy2:  {strat2:.2f} %')

Buy and Hold:   258.09 %   Strategy:  2711.04 %    Strategy2:  452.44 %


In [24]:
# result after publication in 2019
price_w19ff = price_w[price_w.index>'2019-01-01']
pnl_a, buh_a, strat_a, strat2_a = pnl_calculation(price_w19ff)
title = 'PnL 16 weeks pattern vs. b&h, orig. strategy with Instrument: DAX [2019-2024]'
pnl_plotting(pnl_a, title)
print(f'Buy and Hold:   {buh_a:.2f} %   Strategy:  {strat_a:.2f} %    Strategy2:  {strat2_a:.2f} %')

Buy and Hold:   164.68 %   Strategy:  117.33 %    Strategy2:  118.69 %


# 4. Modification on modified Original Strategy

*   identical to the slightly modified Original Strategy of Gebert (except tradetime)
*   tradetime for opening positions (buy/short) & closing positions (sell/cover): each monday 9:00 (at open)



In [41]:
# transforming daily into weekly data
price_w1 = transforming_daily_weekly_open(price)
# other parameters
pos = pos_original
shift_weeks = 12
## weekly trading positions
pos_weekly(price_w1, pos, shift_weeks)

In [42]:
# check if 2018-11-12 is in week_type 4
price_w1[price_w1.index >'2018-11-01'].head(3)

,price,week_nb,week_type,pos
start_w,,,,
2018-11-05,11522.330078,983,3,0
2018-11-12,11591.589844,984,4,0
2018-11-19,11384.000000,985,5,0


In [43]:
pnl1, buh1, strat1, strat12 = pnl_calculation(price_w1)
title='PnL 16 weeks pattern vs. b&h, mod.strategy (1) with Instrument: DAX [2000-2024]'
pnl_plotting(pnl1, title)
print(f'Buy and Hold:   {buh1:.2f} %   Strategy:  {strat1:.2f} %    Strategy2:  {strat12:.2f} %')

Buy and Hold:   249.89 %   Strategy:  3169.90 %    Strategy2:  466.97 %


In [44]:
# result after publication
price_w1_19ff = price_w1[price_w1.index>'2019-01-01']
pnl1_a, buh1_a, strat1_a, strat12_a = pnl_calculation(price_w1_19ff)
title = 'PnL 16 weeks pattern vs. b&h, orig. strategy with Instrument: DAX [2019-2024]'
pnl_plotting(pnl1_a, title)
print(f'Buy and Hold:   {buh1_a:.2f} %   Strategy:  {strat1_a:.2f} %    Strategy2:  {strat12_a:.2f} %')

Buy and Hold:   166.03 %   Strategy:  137.73 %    Strategy2:  135.05 %


In [ ]:
# export of results to excel file
with pd.ExcelWriter(r'/content/gdrive/MyDrive/Colab Notebooks/Weekly Pattern/16_weeks_cycle.xlsx') as writer:
     price.to_excel(writer, sheet_name="daily data")
     price_w1.to_excel(writer, sheet_name="weekly data")
     pnl1.to_excel(writer, sheet_name="PnL")

#5. Sensitivity Analyses

#### Test how sensible the results are regarding shifting the pattern within the 16 weeks cycle

In [45]:
# preparation of results dataframe
result = pd.DataFrame(columns=['shift_wks','buyhold','strategy','strategy2'])
# transforming daily into weekly data
price_w2 = transforming_daily_weekly_open(price)
# calculations
pos = pos_original
weeks = 16
for shift_weeks in range (weeks):
  # calculation of week-type and resulting positions depending on the shift
  pos_weekly(price_w2, pos, shift_weeks)
  pnl, buh, strat, strat2 = pnl_calculation(price_w2)
  result.loc[len(result.index)] = [shift_weeks, buh, strat, strat2]

In [46]:
# plotting results
fig = px.bar(result, x=result['shift_wks'], y=result['strategy'])
fig.update_layout(title='PnL for 16 weeks cycle (various shift parameters)')
fig.add_hline(y=buh, line_dash='dot', line_color='green', annotation_text='Buy and Hold PnL', annotation_position='top left')
fig.update_layout(autosize=False,width=800,height=400)
fig.update_layout(xaxis_title='shift parameter [in wks]', yaxis_title='PnL in %')